In [8]:
from dotenv import load_dotenv
import dspy
from dspy.datasets import HotPotQA
from dspy.evaluate import answer_exact_match
from dspy.evaluate import answer_exact_match
from hotpotqa_agent import hotpotqa_agent

In [9]:
hotpot_qa_ds = HotPotQA(train_size=3000, eval_seed=2025)
hotpot_qa_ds

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'hotpot_qa' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'hotpot_qa' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


In [64]:
load_dotenv(".env")

# Configure DSPY with a language model
lm = dspy.LM("gemini/gemini-2.5-flash", cache=True)
dspy.configure(lm=lm, temperature=0.7)

In [ ]:
question_num = 0
for ex in hotpot_qa_ds.train:
    print(f"Question #{question_num}: {ex['question']}")
    pred = hotpotqa_agent(question=ex["question"])

    if answer_exact_match(ex, pred):
        print("Correct!")
    else:
        print("Incorrect!")

    question_num += 1
    break


Question #0: The city where the Anubis Shrine was found was known to the ancient Egyptians as what?
Correct!


In [98]:
def get_pred_message_responses(pred):
    from dspy.clients.base_lm import GLOBAL_HISTORY

    pred_steps = len(pred.trajectory) // 4

    # get last pred_step number of convo turns
    message_responses = []
    for pred_step in range(pred_steps):
        history_step_index = 10 - (pred_steps - pred_step) - 1
        # first part of messages + response is messages
        pred_step_messages_response = GLOBAL_HISTORY[history_step_index][
            "messages"
        ]  # go back in history to the pred_step of interest
        # add the response
        response = (
            GLOBAL_HISTORY[history_step_index]["response"].choices[0].message.content
        )
        formatted_response = {"role": "assistant", "content": response}
        pred_step_messages_response.append(formatted_response)

        message_responses.append(pred_step_messages_response)

    return message_responses


get_pred_message_responses(pred)

[[{'role': 'system',
   'content': 'Your input fields are:\n1. `question` (str): \n2. `trajectory` (str):\nYour output fields are:\n1. `next_thought` (str): \n2. `next_tool_name` (Literal[\'evaluate_math\', \'search_wikipedia\', \'search_web\', \'finish\']): \n3. `next_tool_args` (dict[str, Any]):\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\nInputs will have the following structure:\n\n[[ ## question ## ]]\n{question}\n\n[[ ## trajectory ## ]]\n{trajectory}\n\nOutputs will be a JSON object with the following fields.\n\n{\n  "next_thought": "{next_thought}",\n  "next_tool_name": "{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: evaluate_math; search_wikipedia; search_web; finish",\n  "next_tool_args": "{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {\\"type\\": \\"object\\", \\"additionalProperties\\": true}"\n}\nIn adhering to this structu